In [1]:
##Libraries
#Dataframe & Arrays
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv('../data/cleaned_df2.csv')

In [3]:
df = df.sort_values(by=['StockCode','UnitPrice'], ascending=False)

In [5]:
df['B2B'] = 1

In [6]:
df_pricing_file = df.groupby(['B2B','StockCode','UnitPrice']).agg({'Rev': ['sum','count'],'Quantity':['sum']}).reset_index()
df_pricing_file = df_pricing_file.sort_values(by=['StockCode','UnitPrice'], ascending=False)
df_pricing_file.head()

B2B StockCode UnitPrice    Rev       Quantity
                                sum count      sum
7287   1    90214Z      0.29   3.48     1       12
7286   1    90214Y      1.25   1.25     1        1
7285   1    90214Y      0.29  17.40     2       60
7284   1    90214W      0.29   3.48     1       12
7283   1    90214V      1.25   2.50     2        2

In [7]:
df_pricing_file.columns = ['B2B','StockCode','UnitPrice','Rev','Count','Qty']
df_pricing_file.head()

,B2B,StockCode,UnitPrice,Rev,Count,Qty
7287,1,90214Z,0.29,3.48,1,12
7286,1,90214Y,1.25,1.25,1,1
7285,1,90214Y,0.29,17.40,2,60
7284,1,90214W,0.29,3.48,1,12
7283,1,90214V,1.25,2.50,2,2


In [8]:
df_awp = df_pricing_file.groupby(['B2B','StockCode']).agg({'Rev': ['sum'],'Qty':['sum']})
df_awp.columns = ['Rev','Qty']

In [9]:
df_awp.head()

Rev  Qty
B2B StockCode             
1   10002      699.55  823
    10080      114.41  291
    10120       40.53  193
    10123C       3.25    5
    10124A       6.72   16

In [10]:
def AWP(row):
    return (row['Rev'] / row['Qty'])

In [11]:
df_awp['AWP'] = (df_awp['Rev'] / df_awp['Qty'])
df_awp.head()

Rev  Qty       AWP
B2B StockCode                       
1   10002      699.55  823  0.850000
    10080      114.41  291  0.393162
    10120       40.53  193  0.210000
    10123C       3.25    5  0.650000
    10124A       6.72   16  0.420000

In [12]:
df_awp.reset_index(inplace=True)
df_awp.columns  = ['B2B','StockCode','Rev','Qty','AWP']
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP
0,1,10002,699.55,823,0.850000
1,1,10080,114.41,291,0.393162
2,1,10120,40.53,193,0.210000
3,1,10123C,3.25,5,0.650000
4,1,10124A,6.72,16,0.420000


In [ ]:
## trying to identify mode of Qty purchased by StockCode

In [13]:
df_qty_mode = df.groupby(['B2B','StockCode'])['Quantity'].mean().astype(int).to_frame()
df_qty_mode.reset_index(inplace=True)
df_qty_mode.columns  = ['B2B','StockCode','Quantity']

In [14]:
df_qty_mode.head()

,B2B,StockCode,Quantity
0,1,10002,16
1,1,10080,13
2,1,10120,6
3,1,10123C,1
4,1,10124A,3


In [15]:
def qtymodelookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding mode of quantity sold for that item
    
    The purpose of this funtion is to vlookup value of item from a reference dataframe (df_qty_mode)
    """
    try:
        return int(df_qty_mode.loc[df_qty_mode['StockCode'] == row]["Quantity"])
    except:
        #print(f"Failed on row {row}")
        pass

In [16]:
df_awp['ModeQty'] = df_awp['StockCode'].apply(qtymodelookup)

In [17]:
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP,ModeQty
0,1,10002,699.55,823,0.850000,16
1,1,10080,114.41,291,0.393162,13
2,1,10120,40.53,193,0.210000,6
3,1,10123C,3.25,5,0.650000,1
4,1,10124A,6.72,16,0.420000,3


In [ ]:
df_awp.to_csv('../data/df_awp2.csv', index=True)